In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt

x_train_data=np.load('/content/drive/MyDrive/mnist.npz (Unzipped Files)/x_train.npy')
y_train_data=np.load('/content/drive/MyDrive/mnist.npz (Unzipped Files)/y_train.npy')
x_test_data=np.load('/content/drive/MyDrive/mnist.npz (Unzipped Files)/x_test.npy')
y_test_data=np.load('/content/drive/MyDrive/mnist.npz (Unzipped Files)/y_test.npy')

In [3]:
x_train_data_vector=x_train_data.reshape(x_train_data.shape[0],-1)
x_test_data_vector=x_test_data.reshape(x_test_data.shape[0],-1)
print(x_train_data_vector)
print(x_train_data_vector.shape)
print(x_test_data_vector.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(60000, 784)
(10000, 784)


In [4]:
num_of_samples=[]
sum=0
sum_test=0
num_test=[]
for cls in range(3):
  num = np.sum(y_train_data == cls)
  sum+=num
  num_of_samples.append(num)
for cls in range(3):
  num = np.sum(y_test_data == cls)
  sum_test+=num
  num_test.append(num)
print(sum_test)
print(num_of_samples)

3147
[5923, 6742, 5958]


In [5]:
data_matrix = np.empty((784, sum))
# test_data_matrix=np.empty((784,sum_test))
y_test_labels=[]

y_train_labels = []
print(data_matrix.size)
temp_x=[]
temp_y=[]

temp_test_x=[]
temp_test_y=[]

for i in range(len(x_train_data_vector)):
  if(y_train_data[i]==0 or y_train_data[i]==1 or y_train_data[i]==2):
    temp_x.append(x_train_data_vector[i])
    temp_y.append(y_train_data[i])
temp_x=np.array(temp_x)
temp_y=np.array(temp_y)

for i in range(len(x_test_data_vector)):
  if(y_test_data[i]==0 or y_test_data[i]==1 or y_test_data[i]==2):
    temp_test_x.append(x_test_data_vector[i])
    temp_test_y.append(y_test_data[i])
temp_test_x=np.array(temp_test_x)
temp_test_y=np.array(temp_test_y)

# print(temp.shape)

# for i in range(3):
#     sample_idx = np.where(y_train_data == i)[0]
#     y_train_labels.extend([i] * len(sample_idx))
#     for j, idx in enumerate(sample_idx):
#         sample = np.reshape(x_train_data[idx], (784, 1))
#         data_matrix[:, i * len(sample_idx) + j] = sample[:, 0]
#         print()
# for i in range(3):
#     sample_idx = np.where(y_test_data == i)[0]
#     y_test_labels.extend([i] * len(sample_idx))
#     for j, idx in enumerate(sample_idx):
#         sample = np.reshape(x_test_data[idx], (784, 1))
#         test_data_matrix[:, i * len(sample_idx) + j] = sample[:, 0]
# y_test_labels= np.array(y_test_labels)
# y_train_labels = np.array(y_train_labels)

data_matrix=temp_x.copy()
data_matrix = data_matrix.T
print(data_matrix.shape)
y_train_labels=temp_y.copy()
test_data_matrix=(temp_test_x.T).copy()
y_test_labels=temp_test_y.copy()

14600432
(784, 18623)


In [6]:
mean=np.mean(data_matrix, axis=1, keepdims=True)
print(mean.shape)
centralised_data_matrix=data_matrix-mean
print(centralised_data_matrix.shape)

(784, 1)
(784, 18623)


In [7]:
Covariance_matrix=(1/18622)*np.dot(centralised_data_matrix, np.transpose(centralised_data_matrix))
eigen_values,eigen_vectors=np.linalg.eigh(Covariance_matrix)
sorted_indices = np.argsort(eigen_values)[::-1]
sorted_eigen_values = eigen_values[sorted_indices]
U=sorted_eigen_vectors= eigen_vectors[:, sorted_indices]
sorted_means = mean[sorted_indices]

In [8]:
Y=np.dot(U.T,centralised_data_matrix)
reconstruct_data_matrix=np.dot(U,Y)
print(Y.shape)
MSE=0
for i in range(784):
  for j in range(1000):
    MSE+=(centralised_data_matrix[i][j]-reconstruct_data_matrix[i][j])**2
print(MSE)

(784, 18623)
1.041797091827179e-20


In [9]:
p=0
print(U.shape)
print(Y.shape)
resulting_accuracy=[]

# initial_probs_=[]
# initial_probs_.extend([1/10]*10)
# print(initial_probs_)
p_list=[10]
for p in p_list:
  Up=U[:,:p]
  print(Up.shape)
  Yp=np.dot(Up.T,centralised_data_matrix)
  reconstruct_data_matrix = np.dot(Up,Yp)
  # centralised_x_test=
  Y_p_test=np.dot(Up.T,test_data_matrix-mean)

  print("Yp Size",Yp.shape)
  # print(Yp)
  print(Yp[0][1])

temp=Yp.copy()
lab=y_train_labels.copy()

(784, 784)
(784, 18623)
(784, 10)
Yp Size (10, 18623)
-870.0186208917653


In [10]:
def get_max_class(count):
  if(count[0]>count[1] and count[0]> count[2]):
    return 0
  elif(count[1]>count[0] and count[1]>count[2]):
    return 1
  else:
    return 2

In [11]:
def Model_Decision_tree(train_x, train_y):
  print(train_x.shape[0])
  optimal_split_point = None
  optimal_split_dimension = None
  min_gini = float('inf')
  fl=0
  optimal_left_index=[]
  optimal_right_index=[]
  for p in range(train_x.shape[1]):

    # Calulating Midpoint and splitting it

    min_value = np.min(train_x[:, p])
    print("P: ", p)
    print("Min value: ", min_value)
    max_value = np.max(train_x[:, p])
    print("Max value: ", max_value)

    # sum=0
    # for i in range(train_x.shape[0]):
    #   sum+=train_x[i][p]
    # midpoint=sum/train_x.shape[0]


    midpoint = (min_value + max_value) / 2


    # left_index = train_x[:, p] < midpoint
    # left_index = np.where(train_x[:, p] < midpoint)[0]

    left_index=[]
    right_index=[]


    for i in range(train_x.shape[0]):
      if (train_x[i][p]<midpoint):
        left_index.append(i)
      else:
        right_index.append(i)
    print(p)
    count=[0,0,0]
    for j in optimal_left_index:
      count[train_y[j]]+=1
    print(count)
    class_left=get_max_class(count)
    count=[0,0,0]
    for j in optimal_right_index:
      count[train_y[j]]+=1
    print(count)
    class_right=get_max_class(count)
    # for i in train_x[:,p]:

    # for i in range(train_x.shape[0]):
      # print("left values: ",left_index)
    # right_index = train_x[:,p]>=midpoint
    # right_index = np.where(train_x[:, p] >= midpoint)[0]

    y_l=train_y[left_index]
    y_r=train_y[right_index]

    # Calculating Gini Index

    left_region_gini=0
    right_region_gini=0
    samples=len(y_l)+len(y_r)
    prob_left=len(y_l)/samples
    prob_right=len(y_r)/samples

    for cls in range(3):
      prob_left_region=np.sum(y_l == cls )/len(y_l)
      left_region_gini+=prob_left_region*(1-prob_left_region)

    for cls in range(3):
      prob_right_region=np.sum(y_r == cls )/len(y_r)
      right_region_gini+=prob_right_region*(1-prob_right_region)

    # for i in range(3):
    #   prob_left_region=

    gini=prob_left*left_region_gini + prob_right*right_region_gini
    print("left values: ",left_index)
    print("Len_Left, ", len(left_index))
    print("Len_Right, ", len(right_index))
    # fl=0
    print("Left region gini: ",left_region_gini)
    print("Right region gini: ", right_region_gini)


    # Optimal_Value:
    if(gini<min_gini):
      print("####",p,gini,min_gini)
      min_gini=gini
      optimal_split_dimension=p
      optimal_split_point=midpoint
      optimal_left_index=left_index
      optimal_right_index=right_index
      print(optimal_split_dimension,optimal_split_point)
      if(left_region_gini>right_region_gini):
        fl=0 #Second split will be along left
      else:
        fl=1 #Second split will be along right
      # return  optimal_left_index, optimal_right_index, optimal_split_point, optimal_split_dimension, min_gini, fl

  # left_index = train_x[:, p] < optimal_split_point
  # right_index = train_x[:,p]>=optimal_split_point
  # left_index = np.where(train_x[:, p] < optimal_split_point)[0]
  # right_index = np.where(train_x[:, p] >= optimal_split_point)[0]
  print(print("op_left values: ",train_x[optimal_left_index]))
  # print(left_index)
  # print(right_index)
  count=[0,0,0]
  for j in optimal_left_index:
    count[train_y[j]]+=1
  print(count)
  count=[0,0,0]
  for j in optimal_right_index:
    count[train_y[j]]+=1
  print(count)


  return class_left,class_right,optimal_left_index, optimal_right_index, optimal_split_point, optimal_split_dimension, min_gini, fl

In [12]:
def Predict_Decision_tree(test_values, dim, mid, flag, classes):
  predictions = []
  print(flag)
  for sample in test_values:
    if flag: #left is not split further
      if sample[dim[0]] < mid[0]:
        predictions.append(classes[0])
      else:
        if sample[dim[1]] < mid[1]:
          predictions.append(classes[1])
        else:
          predictions.append(classes[2])
    else: #flag==1 right is not split further
      if sample[dim[0]] >= mid[0]:
        predictions.append(classes[0])
      else:
        if sample[dim[1]] < mid[1]:
          predictions.append(classes[1])
        else:
          predictions.append(classes[2])
  return predictions

In [13]:
flag_left_right=0
classes=[]
mid_pts=[]
dims=[]
class_left_one,class_right_one,left_index_level_one, right_index_level_one, optimal_split_point_level_one, optimal_split_dimension_level_one, min_gini_level_one, flag_left_right=Model_Decision_tree(temp.T, lab)
print("after split_1 len left: ",len(left_index_level_one))
print("after split 1 len right: ",len(right_index_level_one))
print("Lvl-1 Optimal_split_Dim:",optimal_split_dimension_level_one)
print("Lvl-1 Optimal_split_val:",optimal_split_point_level_one)
print("Lvl-1 Gini: ",min_gini_level_one)
print("PLEASE GOD LET's GOOOOOO!!")
count=[0,0,0]
for j in left_index_level_one:
  count[lab[j]]+=1
print(count)
count=[0,0,0]
for j in right_index_level_one:
  count[lab[j]]+=1
print(count)


count=[0,0,0]
flag=flag_left_right
classes=[0,0,0]

# left_index_level_two_l, right_index_level_two_l, optimal_split_point_level_two_l,optimal_split_dimension_level_two_l, min_gini_level_two_l = Model_Decision_tree(temp.T[left_index_level_one], lab[left_index_level_one])
# left_index_level_two_r, right_index_level_two_r, optimal_split_point_level_two_r,optimal_split_dimension_level_two_r, min_gini_level_two_r = Model_Decision_tree(temp.T[right_index_level_one], lab[right_index_level_one])
flag_left_right_2=0
if(flag_left_right==0):
  class_left_2,class_right_2,left_index_level_two, right_index_level_two, optimal_split_point_level_two,optimal_split_dimension_level_two, min_gini_level_two, flag_left_right_2=Model_Decision_tree(temp.T[left_index_level_one], lab[left_index_level_one])
  # print("Second Split comes from left of first.")
  print(len(left_index_level_two),len(right_index_level_two))
  # for i in range(3):
  #   count.append(np.sum(lab[right_index_level_one] == i ))
  # classes[0]=get_max_class(count)
  # print("CLASS[0]: ", classes[0])
  # print("COUNT: ",count)
  # print(class_left_one,class_right_one,class_left_2,class_right_2)
  classes=[class_right_one,class_left_2,class_right_2]
  mid_pts=[optimal_split_point_level_one,optimal_split_point_level_two]
  dims=[optimal_split_dimension_level_one,optimal_split_dimension_level_two]
  print(classes)
  
else:
  class_left_2,class_right_2,left_index_level_two, right_index_level_two, optimal_split_point_level_two,optimal_split_dimension_level_two, min_gini_level_two, flag_left_right_2=Model_Decision_tree(temp.T[right_index_level_one], lab[right_index_level_one])
  # print("Second Split comes from right of first.")
  classes=[class_left_one,class_left_2,class_right_2]
  mid_pts=[optimal_split_point_level_one,optimal_split_point_level_two]
  dims=[optimal_split_dimension_level_one,optimal_split_dimension_level_two]
  print(classes)


18623
P:  0
Min value:  -1153.003205954931
Max value:  2309.4696590235226
0
[0, 0, 0]
[0, 0, 0]
left values:  [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 14, 17, 19, 20, 22, 23, 25, 26, 27, 32, 33, 34, 35, 36, 37, 38, 39, 41, 44, 46, 47, 48, 49, 50, 51, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 71, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 140, 143, 144, 145, 146, 147, 150, 151, 152, 156, 157, 158, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 175, 176, 177, 178, 179, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 195, 196, 197, 199, 202, 203, 205, 206, 208, 209, 211, 212, 213, 214, 216, 218, 220, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 234, 237, 238, 239, 240, 241, 242, 243, 245, 248, 249, 251, 252, 253, 255, 258, 259, 260, 261, 262, 264, 266, 267, 

In [15]:
predictions=Predict_Decision_tree(Y_p_test.T, dims, mid_pts,flag_left_right,classes)
print(predictions)
print(lab)
class_total_samples = np.zeros(3)
class_correct_predictions = np.zeros(3)

for i in range(len(y_test_labels)):
    class_total_samples[y_test_labels[i]] += 1
    if predictions[i] == y_test_labels[i]:
        class_correct_predictions[y_test_labels[i]] += 1

accuracy = np.sum(class_correct_predictions) / np.sum(class_total_samples)

class_wise_accuracy = class_correct_predictions / class_total_samples

print("Overall Accuracy:", accuracy)
print("Class wise Accuracy:", class_wise_accuracy)

0
[2, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 2, 1, 1, 1, 0, 0, 1, 2, 2, 1, 0, 1, 1, 1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 2, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 0, 2, 1, 1, 2, 1, 0, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 1, 2, 1, 2, 2, 0, 1, 1, 0, 0, 2, 2, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 1, 0, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 0, 0, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 1, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 2, 1, 2, 0, 2, 2, 2, 2, 1, 1, 2, 0, 0, 1, 0, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 0, 0, 2, 1, 1, 2, 1